# Deploy a un endpoint online

## Conectar a workspace

In [1]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

Found the config file in: /config.json


Conectado al Workspace: naturgyml4


## Definir y crear un endpoint

Definir el endpoint. 
> Tener en cuenta que el nombre debe ser único: Usaremos la función `dateTime` para ello.

In [2]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

# online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

online_endpoint_name = 'my-diabetes-endpoint-ric'

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow diabetes model",
    auth_mode="key",
)

Crear el endpoint:

In [3]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://my-diabetes-endpoint-ric.northcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://my-diabetes-endpoint-ric.northcentralus.inference.ml.azure.com/swagger.json', 'name': 'my-diabetes-endpoint-ric', 'description': 'Online endpoint for MLflow diabetes model', 'tags': {}, 'properties': {'createdBy': 'Ricardo Ahumada', 'createdAt': '2025-05-27T08:50:17.219280+0000', 'lastModifiedAt': '2025-05-27T08:50:17.219280+0000', 'azureml.onlineendpointid': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/providers/microsoft.machinelearningservices/workspaces/naturgyml4/onlineendpoints/my-diabetes-endpoint-ric', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/oeidp:e4ba3a3a-3776-4c8a-981f-845d8977c1a2:4

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> ¡IMPORTANTE!¡Esperar hasta que el endpoint se cree con éxito antes de continuar!
Una notificación verde debería aparecer en el estudio. </p>



## Configurar el deployment

### Modelo local

In [4]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# Alternativa: Obtener el modelo de la ruta local
model = Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model",
    name="mlflow-diabetes-model",
)

In [5]:
# definir el deployment blue
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
    app_insights_enabled=True  # Habilitar Application Insights
)

In [6]:
# Crear el deployment
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Instance type STANDARD_DS1_V2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint my-diabetes-endpoint-ric exists


.....................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'my-diabetes-endpoint-ric', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/odidp:e4ba3a3a-3776-4c8a-981f-845d8977c1a2:f755b63e-598c-4156-b940-8691a1000e19?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/onlineEndpoints/my-diabetes-endpoint-ric/deployments/blue', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ricardoenm2/code/Users/formacion/azure-machine-learning/3.Despliegue_Modelos/labs/lab2', 'creation_context': <azure.ai.

In [ ]:
# registrar el modelo en el workspace
# ml_client.models.create_or_update(model)

### Modelo registrado

In [ ]:
reg_model=ml_client.models.get(name="mlflow-diabetes-model", version="1")
reg_model.id

In [ ]:
from azure.ai.ml.entities import Environment

# Definir un entorno personalizado
env = Environment(
    name="my-custom-env",
    description="Entorno personalizado",
    image="mcr.microsoft.com/azureml/mlflow-ubuntu18.04-py38-cpu-inference:latest",
    conda_file='./model/conda.yaml',
)

# Registrar el entorno
registered_env = ml_client.environments.create_or_update(env)
print(f"Entorno registrado: {registered_env.name}, Versión: {registered_env.version}")

In [ ]:
curated_envs = ml_client.environments.list()
for env in curated_envs:
    print(f"Nombre: {env.name}, Versión: {env.version}")

In [ ]:
# env = ml_client.environments.get(name="my-custom-env", version="1")
# env

In [ ]:
# definir un nuevo deployment con el entorno personalizado
green_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=online_endpoint_name,
    model=reg_model.id,
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
    # environment=env.id,
)

In [ ]:
# Crear el deployment
ml_client.online_deployments.begin_create_or_update(green_deployment).result()

Asignar el % de tráfico.

In [7]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
# endpoint.traffic = {"green": 20}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://my-diabetes-endpoint-ric.northcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://my-diabetes-endpoint-ric.northcentralus.inference.ml.azure.com/swagger.json', 'name': 'my-diabetes-endpoint-ric', 'description': 'Online endpoint for MLflow diabetes model', 'tags': {}, 'properties': {'createdBy': 'Ricardo Ahumada', 'createdAt': '2025-05-27T08:50:17.219280+0000', 'lastModifiedAt': '2025-05-27T09:02:25.849661+0000', 'azureml.onlineendpointid': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/providers/microsoft.machinelearningservices/workspaces/naturgyml4/onlineendpoints/my-diabetes-endpoint-ric', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/oeidp:e4ba3a3a-3776-4c8a-981f-845d8977c1a2:3

## Probar el deploymnent

In [13]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

Diabetic


## Listar los endpoints

In [14]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

my-diabetes-endpoint-ric
my-deployment-endpoint-ric


## Obtener detalles de un endpoint

In [15]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://my-diabetes-endpoint-ric.northcentralus.inference.ml.azure.com/score


## Borrar el endpoint y deployment

In [ ]:
# Eliminar un deployment
ml_client.online_deployments.begin_delete(
    name='green',
    endpoint_name=online_endpoint_name
).result()

In [ ]:
# Eliminar el endpoint
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)